In [ ]:
from surface_velocity.readers import read_HDF5_ATL03, get_ATL03_x_atc

atl3_dir = '/Users/benhills/Google Drive File Stream/Shared drives/IceSat2_Surface_Velocity/Shared_Data/FIS_0848_ATL03/ATL03_files/'
fn_1 = atl3_dir + '177516372/ATL03_20190822153035_08480411_003_01.h5'
fn_2 = atl3_dir + '177261775/ATL03_20190523195046_08480311_003_01.h5'

# read the IS2 data with Tyler's ATL03 reader:
IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams = read_HDF5_ATL03(fn_1)
# add x_atc to the ATL03 data structure (this function adds to the LS2_ATL03_mds dictionary)
get_ATL03_x_atc(IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams)
#-- select the 2l beam from ATL03
D3 = IS2_atl03_mds['gt2l']
LMH=D3['heights']['signal_conf_ph'][:,3] >= 2
x3_1 = D3['heights']['x_atc'][LMH]
h3_1 = D3['heights']['h_ph'][LMH]

# read the IS2 data with Tyler's ATL03 reader:
IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams = read_HDF5_ATL03(fn_2)
# add x_atc to the ATL03 data structure (this function adds to the LS2_ATL03_mds dictionary)
get_ATL03_x_atc(IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams)
#-- select the 2l beam from ATL03
D3 = IS2_atl03_mds['gt2l']
LMH=D3['heights']['signal_conf_ph'][:,3] >= 2
x3_2 = D3['heights']['x_atc'][LMH]
h3_2 = D3['heights']['h_ph'][LMH]

In [ ]:
from surface_velocity.atl03_reprocessing import reinterpolate_atl03

h3_1 = h3_1[np.logical_and(x3_1>np.nanmin(D1['x_atc']),x3_1<np.nanmax(D1['x_atc']))]
x3_1 = x3_1[np.logical_and(x3_1>np.nanmin(D1['x_atc']),x3_1<np.nanmax(D1['x_atc']))]
h3_2 = h3_2[np.logical_and(x3_2>np.nanmin(D2['x_atc']),x3_2<np.nanmax(D2['x_atc']))]
x3_2 = x3_2[np.logical_and(x3_2>np.nanmin(D2['x_atc']),x3_2<np.nanmax(D2['x_atc']))]

#!!!!!!!!!!!!!!!!!!! This takes forever, instead of reprocessing just reload the numpy arrays in the next cell
"""
x_3_1,h_3_1 = reinterpolate_atl03(x3_1,h3_1)
x_3_2,h_3_2 = reinterpolate_atl03(x3_2,h3_2)

np.save('../data/reinterpret_atl03_file1.npy',np.array([x_3_1,h_3_1]))
np.save('../data/reinterpret_atl03_file2.npy',np.array([x_3_2,h_3_2]))
"""

In [ ]:
atl3_dir = '/Users/benhills/Google Drive File Stream/Shared drives/IceSat2_Surface_Velocity/Shared_Data/FIS_0848_ATL03/reinterpolated/'
x1,h1 = np.load(atl3_dir+'reinterpolated_atl03_file1.npy')
x2,h2 = np.load(atl3_dir+'reinterpolated_atl03_file2.npy')

h1_smooth,dh1 = smooth_and_diff(x1,h1,win=100)
h2_smooth,dh2 = smooth_and_diff(x2,h2,win=100)

vel_xs = np.linspace(np.min(x1)+1000,np.max(x1)-1000,1000)
velocities,correlations = velocity(x1,dh1,dh2,dt,vel_xs,search_width=1000,segment_length=5000,dx=5)

# ------------------------------------------

plt.figure(figsize=(8,4))
gs = GridSpec(2,2)

plt.subplot(gs[0,0])
plt.tick_params(bottom=False,labelbottom=False)
plt.plot(x1/1000.-29000,h1,'.',c='indianred')
plt.plot(x2/1000.-29000,h2,'.',c='steelblue',ms=3)
plt.ylabel('Elevation (m)')
plt.title('ATL03',fontweight='bold')
plt.xlim(80,580)

plt.subplot(gs[1,0])
plt.tick_params(bottom=False,labelbottom=False)
plt.plot(x1/1000.-29000,dh1,'.',c='indianred')
plt.plot(x2/1000.-29000,dh2,'.',c='steelblue',ms=3)
plt.ylim(-.05,.05)
plt.ylabel('Surface Slope (m/m)')
plt.xlim(80,580)

ax5 = plt.subplot(gs[:,1])
plt.plot(vel_xs/1000.-29000,velocities,'.',c='c',mfc='w',label='ATL06')
plt.ylabel('Velocity (m/yr)')
plt.xlabel('Along-Track Distance (km)')
plt.xlim(80,580)
plt.ylim(-500,1500)

plt.tight_layout()